In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

print("Bibliotecas importadas.")

In [ ]:
#Célula 2.1: Visualização Comparativa Acelerômetro
caminho_suave = 'data/suave/suave_1.csv'
caminho_normal = 'data/normal/normal_1.csv'
caminho_agressivo = 'data/agressivo/agressivo_1.csv'

df_suave = pd.read_csv(caminho_suave, sep = ';', decimal = ',', skiprows = 1)
df_normal = pd.read_csv(caminho_normal, sep = ';', decimal = ',', skiprows = 1)
df_agressivo = pd.read_csv(caminho_agressivo, sep = ';', decimal = ',', skiprows = 1)

#Gráfico com duas subtramas
fig, ax = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
fig.suptitle('Comparação de Aceleração (Magnitude) entre Percursos', fontsize=16)

# Calcula a magnitude da aceleração para cada percurso
df_suave['acel_magnitude'] = np.sqrt(df_suave['ax']**2 + df_suave['ay']**2)
df_normal['acel_magnitude'] = np.sqrt(df_normal['ax']**2 + df_normal['ay']**2)
df_agressivo['acel_magnitude'] = np.sqrt(df_agressivo['ax']**2 + df_agressivo['ay']**2)

#Plot do percurso suave
ax[0].plot(df_suave['time'], df_suave['acel_magnitude'], color='green')
ax[0].set_title('Percurso Suave')
ax[0].set_ylabel('Aceleração (m/s^2)')
ax[0].grid(True)

#Plot do percurso normal
ax[1].plot(df_normal['time'], df_normal['acel_magnitude'], color='orange')
ax[1].set_title('Percurso Normal')
ax[1].set_ylabel('Aceleração (m/s^2)')
ax[1].grid(True)

#Plot do percurso aggressivo
ax[2].plot(df_agressivo['time'], df_agressivo['acel_magnitude'], color='red')
ax[2].set_title('Percurso Agressivo')
ax[2].set_xlabel('Tempo (s)')
ax[2].set_ylabel('Aceleração (m/s^2)')
ax[2].grid(True)

plt.show()

In [ ]:
#Célula 2.2: Visualização Comparativa Giroscópio

caminho_suave = 'data/suave/suave_1.csv'
caminho_normal = 'data/normal/normal_1.csv'
caminho_agressivo = 'data/agressivo/agressivo_1.csv'

df_suave = pd.read_csv(caminho_suave, sep = ';', decimal = ',', skiprows = 1)
df_normal = pd.read_csv(caminho_normal, sep = ';', decimal = ',', skiprows = 1)
df_agressivo = pd.read_csv(caminho_agressivo, sep = ';', decimal = ',', skiprows = 1)

#Gráfico com três subtramas
fig, ax = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

fig.suptitle('Comparação de Giroscópio (Intensidade das Curvas) entre Percursos', fontsize=16)

# Plot do percurso suave
ax[0].plot(df_suave['time'], df_suave['wz'].abs(), color='green')
ax[0].set_title('Percurso Suave')
ax[0].set_ylabel('Veloc. Angular |wz| (rad/s)')
ax[0].grid(True)

# Plot do percurso normal
ax[1].plot(df_normal['time'], df_normal['wz'].abs(), color='orange')
ax[1].set_title('Percurso Normal')
ax[1].set_ylabel('Veloc. Angular |wz| (rad/s)')
ax[1].grid(True)

# Plot do percurso aggressivo
ax[2].plot(df_agressivo['time'], df_agressivo['wz'].abs(), color='red')
ax[2].set_title('Percurso Agressivo')
ax[2].set_xlabel('Tempo (s)')
ax[2].set_ylabel('Veloc. Angular |wz| (rad/s)')
ax[2].grid(True)

#Evita sobreposição dos gráficos
plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()

In [ ]:
#Célula 3: Função de Extração de Características 

def extrair_caracteristicas(df_percurso):
    
    #Recebe um DataFrame de um único percurso e retorna um dicionário de características
    #Usa os nomes de coluna 'ax', 'ay', 'wz' do arquivo.

    caracteristicas = {}
    
    #Colunas de aceleração e giroscópio
    acel_cols = ['ax', 'ay']
    giro_cols = ['wz']

    #Calcula a magnitude da aceleração, ignorando possíveis valores nulos
    df_percurso['acel_magnitude'] = np.sqrt(df_percurso[acel_cols[0]]**2 + df_percurso[acel_cols[1]]**2).dropna()

    #Características da Aceleração
    caracteristicas['acel_std'] = df_percurso['acel_magnitude'].std()
    caracteristicas['acel_max'] = df_percurso['acel_magnitude'].max()
    caracteristicas['acel_q95'] = df_percurso['acel_magnitude'].quantile(0.95)

    #Características do Giroscópio
    caracteristicas['giro_std'] = df_percurso[giro_cols[0]].abs().std()
    caracteristicas['giro_max'] = df_percurso[giro_cols[0]].abs().max()
    caracteristicas['giro_q95'] = df_percurso[giro_cols[0]].abs().quantile(0.95)

    return caracteristicas

print("Função definida.")

In [ ]:
#Célula 4: Construção do Dataset de Características

DATA_DIR = 'data'
categorias = ['agressivo', 'normal', 'suave']
lista_de_percursos = []
colunas_necessarias = ['ax', 'ay', 'wz']

print("Processando arquivos...")

for categoria in categorias:
    pasta_categoria = os.path.join(DATA_DIR, categoria)
    
    #Verifica se a pasta da categoria existe
    if not os.path.exists(pasta_categoria):
        print(f"AVISO: Pasta da categoria '{categoria}' não encontrada. Pulando.")
        continue
        
    for nome_arquivo in os.listdir(pasta_categoria):
        if nome_arquivo.endswith('.csv'):
            caminho_completo = os.path.join(pasta_categoria, nome_arquivo)
            
            try:
                df_temp = pd.read_csv(
                    caminho_completo, 
                    sep=';', 
                    decimal=',', 
                    skiprows=1,
                    on_bad_lines='skip' #Ignora linhas mal formatadas
                )
                
                #Verifica se o arquivo tem as colunas necessárias
                if all(col in df_temp.columns for col in colunas_necessarias):
                    #Caso sim, extrai as características
                    features = extrair_caracteristicas(df_temp)
                    features['label'] = categoria    #Rotula
                    lista_de_percursos.append(features)
                else:
                    #Caso não tenha, avisa e pula o arquivo
                    print(f"AVISO: Arquivo {nome_arquivo} pulado (colunas {colunas_necessarias} não encontradas).")
                    
            except Exception as e:
                #Se der qualquer outro erro ao ler o arquivo, avisa e pula
                print(f"ERRO ao processar o arquivo {nome_arquivo}: {e}")

#Cria o dataframe principal com todos os dados processados
dataset = pd.DataFrame(lista_de_percursos)

print("\nDataset de características criado com sucesso!")
display(dataset)

In [ ]:
#Célula 5: Geração de Dados Sintéticos (Data Augmentation)

dataset_original = dataset.copy()
print(f"Tamanho do dataset original: {dataset_original.shape[0]} amostras.")

lista_dados_sinteticos = []

#Parâmetros de geração
n_filhos_por_amostra = 999    #Filhos sintéticos a serem criados para cada amostra original
noise_level = 0.15         #Nível de ruído (0.15 = 15% de variação)

for i in range(n_filhos_por_amostra):
    #Itera sobre cada amostra original
    for index, amostra_original in dataset_original.iterrows():
        nova_amostra_sintetica = {}
        features_originais = amostra_original.drop('label')
        
        #Adiciona ruído a cada característica
        for feature_name, value in features_originais.items():
            #Gera um ruído gaussiano
            #O ruído é proporcional ao valor da característica (ex: 10% do valor)
            #Adicionou-se 1e-6 para evitar erros se o valor for 0
            ruido = np.random.normal(0, noise_level * abs(value) + 1e-6)
            nova_amostra_sintetica[feature_name] = value + ruido
        
        # Adiciona o rótulo original
        nova_amostra_sintetica['label'] = amostra_original['label']
        
        #Adiciona a nova amostra sintética à lista
        lista_dados_sinteticos.append(nova_amostra_sintetica)

#Cria um DataFrame com os dados sintéticos
dataset_sintetico = pd.DataFrame(lista_dados_sinteticos)

#Combina o dataset original com o sintético, mantendo os originais
dataset = pd.concat([dataset_original, dataset_sintetico], ignore_index=True)

print(f"Tamanho do novo dataset (Original + Sintético): {dataset.shape[0]} amostras.")

#Visualizando nova distribuição de classes
print("\nNova distribuição de classes:")
print(dataset['label'].value_counts())

#Observando os clusters de dados criados
sns.scatterplot(data=dataset, x='acel_std', y='giro_max', hue='label', s=20, alpha=0.5)
plt.title("Visualização dos Dados Aumentados")
plt.show()

In [ ]:
#Célula 6: Treinamento e Avaliação do Modelo

print("Iniciando Treinamento e Avaliação do Modelo...")
print(f"Total de amostras no dataset: {dataset.shape[0]}")
print("\nAmostras por classe:")
print(dataset['label'].value_counts())

#Verifica se o dataset não está vazio
if dataset.empty:
    print("ERRO: Dataset vazio. Verifique a pasta 'data'.")
else:
    #Separando as características (X) dos rótulos (y)
    X = dataset.drop('label', axis=1)
    y = dataset['label']

    #Dividindo os dados em treino e teste, test_size=0.25 significa que 25% dos dados vão para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    #Criando e treinando o modelo
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)

    #Fazendo previsões nos dados de teste
    y_pred = modelo.predict(X_test)

    #Avaliando os resultados
    acuracia = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo: {acuracia * 100:.2f}%\n")

    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=categorias))

    #Plotando a Matriz de Confusão
    print("Matriz de Confusão:")
    fig, ax = plt.subplots(figsize=(8, 6))
    ConfusionMatrixDisplay.from_estimator(modelo, X_test, y_test, display_labels=categorias, cmap='Blues', ax=ax)
    plt.title('Matriz de Confusão')
    plt.show()